In [40]:
from diffusers import StableDiffusionPipeline
import torch

from transformers import CLIPProcessor, CLIPModel
from PIL import Image

import os

from sklearn.cluster import KMeans
import numpy as np

In [2]:
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.safety_checker = None

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# Use this function to generate images and save them to a folder

def gen_images(prompt, folder_name):
    for i in range(100):
        image = pipe(prompt).images[0]
        image.save(f"{folder_name}/image_{i+1}.png")
    print("Done generating images")
    return True

In [4]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = clip_model.to(device)

In [5]:
def get_clip_embeddings(images):
  """
    This function takes a list of images as input,
    returning a list of clip embeddings of the images.

    :param images: The input image as a string.
    :return: A list of clip embeddings.
    """
  inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)

  with torch.no_grad():
      image_features = clip_model.get_image_features(**inputs)

  return image_features

In [11]:
def get_embedding_from_folder(folder_name):
    embeddings = []
    for img in os.listdir(folder_name):
        image = Image.open(f"{folder_name}/{img}")
        image = image.convert("RGB")
        embeddings.append(get_clip_embeddings([image]))
    return embeddings

In [28]:
# Get embedding of dog images folder
dog_folder = "dog images"
dog_embeddings = get_embedding_from_folder(dog_folder)

In [17]:
len(dog_embeddings)

100

In [29]:
# Convert the embeddings to numpy array
dog_embeddings = np.array([embedding.cpu().numpy() for embedding in dog_embeddings]).squeeze()

## K-Means Clustering

In [72]:
kmeans_clusters = KMeans(n_clusters=10, init='k-means++').fit(dog_embeddings)

In [73]:
# Get different images from each cluster
def get_images_from_cluster(cluster_number, folder_name):
    cluster_images = []
    for i, label in enumerate(kmeans_clusters.labels_):
        if label == cluster_number:
            cluster_images.append(f"{folder_name}/image_{i+1}.png")
    return cluster_images

In [74]:
# Get images for all the clusters
cluster_images = []
for i in range(10):
    cluster_images.append(get_images_from_cluster(i, dog_folder))

In [75]:
distance = kmeans_clusters.transform(dog_embeddings)

In [76]:
closest_images = []

for cluster_number in range(10):
    cluster_indices = np.where(kmeans_clusters.labels_ == cluster_number)[0]
    
    cluster_distances = distance[cluster_indices, cluster_number]
    
    min_distance_indices = cluster_indices[np.argsort(cluster_distances)[:2]]
    
    closest_image_path = [f"{dog_folder}/image_{idx+1}.png" for idx in min_distance_indices]
    closest_images.append(closest_image_path)

In [77]:
closest_images

[['dog images/image_53.png', 'dog images/image_9.png'],
 ['dog images/image_50.png', 'dog images/image_60.png'],
 ['dog images/image_58.png', 'dog images/image_41.png'],
 ['dog images/image_35.png', 'dog images/image_40.png'],
 ['dog images/image_72.png', 'dog images/image_44.png'],
 ['dog images/image_79.png', 'dog images/image_96.png'],
 ['dog images/image_59.png', 'dog images/image_86.png'],
 ['dog images/image_32.png'],
 ['dog images/image_12.png', 'dog images/image_30.png'],
 ['dog images/image_63.png', 'dog images/image_82.png']]